In [21]:
import tensorflow as tf
import pandas as pd
import os
import sys
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model
from sklearn.model_selection import train_test_split

In [359]:
def word_split(mystring):
    str_split = []
    nltk_stopwords= nltk.corpus.stopwords.words('english')
    for tmp in mystring:
        tmp = tmp.lower()
        punct_token = wordpunct_tokenize(tmp)
        '''
        tmp = re.sub('[^a-zA-Z0-9\s\?\!]+', '', tmp)
        tmp = tmp.replace('!', ' !')
        tmp = tmp.replace('?', ' ?')
        tmp = tmp.split(' ')
        '''
        punct_token = [word for word in punct_token if word not in nltk_stopwords]
        #remove string.punctuation
        punct_token = [word for word in punct_token if word not in string.punctuation]
        '''
        while True:
            if '' not in tmp:
                break
            tmp.remove('')
        while True:
            if 'the' not in tmp:
                break
            tmp.remove('the')
        while True:
            if 'and' not in tmp:
                break
            tmp.remove('and')
        while True:
            if 'of' not in tmp:
                break
            tmp.remove('of')
        while True:
            if 'is' not in tmp:
                break
            tmp.remove('is')
        while True:
            if 'are' not in tmp:
                break
            tmp.remove('are')
        '''
        str_split.append(punct_token)
    return str_split

In [391]:
import nltk
nltk.download('stopwords')
import string
from nltk import wordpunct_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rtes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [392]:
GLOVE_DIR = './'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 200
NUM_LSTM_UNITS = 200
VALIDATION_SPLIT = 0.2

In [393]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.200d.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [394]:
all_data = pd.read_csv("train.csv")
data = all_data['Headline']
label = all_data['Label']
my_split = word_split(data)
print(my_split[10])

['america', 'miserable', 'half', 'residents', 'illinois', 'connecticut', 'wish', 'could', 'move', 'people', 'montana', 'hawaii', 'happy', 'staying', 'put']


In [395]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(my_split)
sequences = tokenizer.texts_to_sequences(my_split)
word_index = tokenizer.word_index
x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [396]:
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i-1] = embedding_vector

In [397]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [398]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
shared_lstm = LSTM(NUM_LSTM_UNITS)
xx = shared_lstm(embedded_sequences)
predictions = Dense(units=1, activation='linear')(xx)
model = Model(inputs=sequence_input, outputs=predictions)

In [399]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(0.001))
model.summary()

Model: "functional_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
embedding_33 (Embedding)     (None, 30, 200)           1862400   
_________________________________________________________________
lstm_34 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 201       
Total params: 2,183,401
Trainable params: 321,001
Non-trainable params: 1,862,400
_________________________________________________________________


In [400]:
x_train, x_test, y_train, y_test = train_test_split(x, label, test_size=0.1)
print(x_train.shape)

(1836, 30)


In [401]:
history = model.fit(x_train, y_train ,batch_size=32 ,epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
57/58 [============================>.] - ETA: 0s - loss: 0.9712

InvalidArgumentError:  indices[17,29] = 9312 is not in [0, 9312)
	 [[node functional_69/embedding_33/embedding_lookup (defined at <ipython-input-401-d9cc275b3f22>:1) ]] [Op:__inference_test_function_146270]

Errors may have originated from an input operation.
Input Source operations connected to node functional_69/embedding_33/embedding_lookup:
 functional_69/embedding_33/embedding_lookup/145790 (defined at C:\Users\rtes\anaconda3\envs\tf\lib\contextlib.py:81)

Function call stack:
test_function


In [ ]:
import matplotlib.pyplot as plt
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss,'r', val_loss, 'b')

In [ ]:
test_data = pd.read_csv("test.csv")
data = test_data['Headline']
label = test_data['Label']
test_split = word_split(data)
sequences = tokenizer.texts_to_sequences(test_split)
x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_pre = model.predict(x)
b = np.arange(1, y_pre.shape[0]+1).reshape(y_pre.shape[0], 1)
y_pre = np.append(b, y_pre, axis=1)

In [ ]:
import csv
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID','Label'])
    writer.writerows(y_pre)